# ***Importing Needed Libraries & Modules***

In [ ]:
import numpy as np
import tensorflow as tf
from numpy.polynomial import Polynomial
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
assert tf.__version__.startswith('2.')

# ***Generating Polynomials of Degree 5***

In [ ]:
# Data Generation for Polynomial Roots
def generate_polynomial_data(num_samples=70000):
    coefficients = np.random.uniform(-10, 10, (num_samples, 6))
    coefficients[:, 5] = np.random.uniform(1, 10, num_samples)  # Ensure x^5 coefficient is never 0

    def find_roots(coefs):
        p = Polynomial(coefs)
        roots = p.roots()
        roots_real_imag = np.hstack((root.real, root.imag) for root in roots)[:10]  # Ensure consistent size
        return roots_real_imag

    roots = np.array([find_roots(coefs) for coefs in coefficients])
    return coefficients, roots

X, Y = generate_polynomial_data()

<ipython-input-32-5d364c398b9f>:9: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  roots_real_imag = np.hstack((root.real, root.imag) for root in roots)[:10]  # Ensure consistent size


# ***Normalization***

In [ ]:
# Normalization
scaler_X = MinMaxScaler(feature_range=(-1, 1))
scaler_Y = MinMaxScaler(feature_range=(-1, 1))
X_normalized = scaler_X.fit_transform(X)
Y_normalized = scaler_Y.fit_transform(Y)

# ***Converting to TensorFlow Tensors***

In [ ]:
# Convert to TensorFlow tensors
X_normalized_tf = tf.constant(X_normalized, dtype=tf.float32)
Y_normalized_tf = tf.constant(Y_normalized, dtype=tf.float32)

# ***Neural Network Architecture***

In [ ]:
# Neural Network Architecture
model = Sequential([
    Dense(64, input_dim=6, activation='relu'),
    Dense(64, activation='relu'),
    Dense(10, activation='linear')  # For 5 roots (real and imaginary parts)
])

# ***Training Function***

In [ ]:
# LMA-like Training Function
def adaptive_sgd_training(model, X_train, Y_train, epochs=10, initial_lr=1e-4, increase_factor=1.5, decrease_factor=0.4):
    optimizer = tf.keras.optimizers.SGD(learning_rate=initial_lr)
    prev_loss = float('inf')

    for epoch in range(epochs):
        with tf.GradientTape() as tape:
            predictions = model(X_train, training=True)
            loss = tf.reduce_mean(tf.keras.losses.MSE(Y_train, predictions))

        if loss < prev_loss:
            optimizer.learning_rate.assign(optimizer.learning_rate * increase_factor)
        else:
            model.set_weights(prev_weights)
            optimizer.learning_rate.assign(optimizer.learning_rate * decrease_factor)
            print(f"Epoch {epoch+1}: Loss increased, reverting weights and reducing learning rate.")
            continue

        gradients = tape.gradient(loss, model.trainable_variables)
        optimizer.apply_gradients(zip(gradients, model.trainable_variables))
        prev_weights = model.get_weights()
        prev_loss = loss

        print(f'Epoch {epoch+1}/{epochs}, Loss: {loss.numpy()}, Learning Rate: {optimizer.learning_rate.numpy()}')

# ***Training & Testing***

In [ ]:
# Train the Model
adaptive_sgd_training(model, X_normalized_tf, Y_normalized_tf, epochs=28)

# Generate and Test on New Data
X_test, Y_test_actual = generate_polynomial_data(30000)
X_test_normalized = scaler_X.transform(X_test)
X_test_normalized_tf = tf.constant(X_test_normalized, dtype=tf.float32)
Y_test_pred_normalized_tf = model.predict(X_test_normalized_tf)
Y_test_pred = scaler_Y.inverse_transform(Y_test_pred_normalized_tf)

Epoch 1/28, Loss: 0.015122589655220509, Learning Rate: 0.00014999999257270247
Epoch 2/28, Loss: 0.015122505836188793, Learning Rate: 0.00022499999613501132
Epoch 3/28, Loss: 0.015122383832931519, Learning Rate: 0.0003375000087544322
Epoch 4/28, Loss: 0.015122201293706894, Learning Rate: 0.0005062500131316483
Epoch 5/28, Loss: 0.015121923759579659, Learning Rate: 0.0007593750488013029
Epoch 6/28, Loss: 0.01512151025235653, Learning Rate: 0.0011390625732019544
Epoch 7/28, Loss: 0.015120891854166985, Learning Rate: 0.0017085939180105925
Epoch 8/28, Loss: 0.015119963325560093, Learning Rate: 0.002562890760600567
Epoch 9/28, Loss: 0.015118582174181938, Learning Rate: 0.0038443361409008503
Epoch 10/28, Loss: 0.015116521157324314, Learning Rate: 0.005766504444181919
Epoch 11/28, Loss: 0.015113462693989277, Learning Rate: 0.008649757131934166
Epoch 12/28, Loss: 0.015108945779502392, Learning Rate: 0.012974635697901249
Epoch 13/28, Loss: 0.015102327801287174, Learning Rate: 0.01946195401251316


<ipython-input-32-5d364c398b9f>:9: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  roots_real_imag = np.hstack((root.real, root.imag) for root in roots)[:10]  # Ensure consistent size


938/938 [==============================] - 1s 2ms/step


# ***Comparing***

In [ ]:
# Compare Actual and Predicted Roots
for i in range(len(X_test)):
    print(f"Polynomial Coefficients: {X_test[i]}")
    print("Actual Roots:    ", ", ".join([f"{Y_test_actual[i][2*j]:+.2f} + {Y_test_actual[i][2*j+1]:+.2f}i" for j in range(5)]))
    print("Predicted Roots: ", ", ".join([f"{Y_test_pred[i][2*j]:+.2f} + {Y_test_pred[i][2*j+1]:+.2f}i" for j in range(5)]))
    print("----------")

Streaming output truncated to the last 5000 lines.
Actual Roots:     -1.15 + -0.43i, -1.15 + +0.43i, +0.31 + +0.00i, +0.60 + -0.61i, +0.60 + +0.61i
Predicted Roots:  -1.45 + -0.22i, -0.87 + +0.41i, +0.29 + -0.51i, +0.76 + +0.03i, +0.39 + +0.31i
----------
Polynomial Coefficients: [-1.48238098  5.36870277 -2.19526689 -6.59580415 -8.63263156  1.68749118]
Actual Roots:     -0.71 + -0.71i, -0.71 + +0.71i, +0.36 + -0.13i, +0.36 + +0.13i, +5.81 + +0.00i
Predicted Roots:  -0.52 + -0.37i, -0.62 + +0.27i, +0.18 + +0.12i, +0.89 + +0.30i, +4.23 + -0.10i
----------
Polynomial Coefficients: [ 3.93491184 -7.94455907 -8.08896958 -4.54685904 -5.57907629  2.01727701]
Actual Roots:     -1.03 + +0.00i, -0.14 + -1.21i, -0.14 + +1.21i, +0.34 + +0.00i, +3.73 + +0.00i
Predicted Roots:  -1.43 + +0.14i, -0.01 + -0.80i, -0.11 + +0.69i, +0.50 + +0.34i, +3.54 + -0.20i
----------
Polynomial Coefficients: [ 2.74234992  9.61738353 -2.40137433 -4.48650807 -5.94338253  9.02875702]
Actual Roots:     -0.58 + -0.71i, -0.

# ***Evaluating The Model Using MSE***

In [ ]:
# Evaluate the Model
mse = np.mean((Y_test_pred - Y_test_actual) ** 2)
print(f"\nTest MSE: {mse}")


Test MSE: 0.13713598887899178
